# Conversion from Einstein A coefficient $A_{21}$ to oscillator strength $f$


The following relation is used in the conversion:

$f_{12}=\frac{g_2}{g_1}\frac{A_{21}}{E_{21}^2}\frac{\hbar m_e c^2}{2 \alpha} $

with the values of the constants taken from CODATA https://physics.nist.gov/cuu/Constants/: 
fine structure constant $\alpha = 7.2973525664\times10^{-3}$;
$\hbar=1.054571800\times10^{-34} \mathrm{J}\cdot \mathrm{s}$;
$m_ec^2=8.18710565\times10^{-14} \mathrm{J}$.
Make sure that $E_{21}$ is expressed in Joules, using the relation: $E = hck$, with $k$ usually expressed in $\mathrm{cm}^{-1}$.

The relation above is based on Eq. 33 in Robert C. Hilborn, *Einstein coefficients, cross sections, f values, dipole moments, and all that*, American Journal of Physics **50**, 982 (1982): https://doi.org/10.1119/1.12937.

$f_{12}=\frac{g_2}{g_1}\frac{2\pi\epsilon_0 m_e c^2}{\omega_{21}^2 e^2} A_{21}$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Function for oscillator strength conversion

In [2]:
fine_structure_constant=7.2973525664e-3
hbar=1.054571800e-34 # J s
m_e_c_squared=8.18710565e-14 #J: mass of electron in energy units

h=6.626070040e-34 #J s
c_cm_per_sec=29979245800 #cm/s for conversion of cm^{-1}
icm_to_J=h*c_cm_per_sec #E = hck, with k in cm^{-1}

constantFactor = 0.5*hbar*m_e_c_squared/fine_structure_constant

def oscillatorStrength(branch="R",J_lower=0,A_coeff=0.0, E_in_icm=0.0, use_multiplicities=True):
    if branch == "R":
        J_upper = J_lower + 1
    elif branch == "P":
        J_upper = J_lower - 1
    elif branch == "Q":
        J_upper = J_lower
    else:
        print "Unknown transition type."
        return None
    
    if use_multiplicities:
        f = ( 2.0*J_upper + 1 )/( 2.0*J_lower + 1 )*A_coeff/(E_in_icm*icm_to_J)**2*constantFactor
    else:
        f = A_coeff/(E_in_icm*icm_to_J)**2*constantFactor
    
    #print J_lower,f,( 2.0*J_upper + 1 )/( 2.0*J_lower + 1 ),A_coeff/(E_in_icm*icm_to_J)**2*constantFactor
    return f


In [3]:
#Test function above for Werner v'=1, R-branch
E_Abgrall = [101457.97,101456.88,101395.25,101271.58,101125.54,100870.70,100577.88]
A_Abgrall = [0.1577E+09,0.1360E+09,0.1283E+09,0.1365E+09,0.4860E+07,0.6175E+08,0.7505E+08]

#print "line\tosc_strength"
f_new_conversion = []
for J in range(7):
    f_new_conversion.append(oscillatorStrength(branch="R", J_lower=J, A_coeff=A_Abgrall[J], E_in_icm=E_Abgrall[J]))
    #print "W1R%d\t%0.5f"%(J,f_new_conversion[J])

## Using python pandas in anticipation of application in full dataset

For the moment, comparisons are performed only for the H$_2$ $R$-branch of the Werner ($v=0-1$) band.

The following compares the data from VizieR and a new evalution using data from the MOLAT database (Abgrall et al.)

In [4]:
#From VizieR website
columns = ["Band", "v_upper", "Branch", "v_lower", "wavelength(A)", "e", "ref", "f", "Gamma", "k" ] 
data=[ 
["W",1,"R",0,985.633709,0.000009,1,0.0240614000,1.2e+09,0.00602255],
["W",1,"R",1,985.644316,0.000004,1,0.0207080000,1.2e+09,0.00604040],
["W",1,"R",2,986.244066,0.000006,1,0.0195224000,1.2e+09,0.00551177],
["W",1,"R",3,987.448680,0.000070,1,0.0207847000,1.2e+09,0.00511604],
["W",1,"R",4,988.871510,0.000040,1,0.0007409790,1.1e+09,0.01493980],
["W",1,"R",5,991.371720,0.000050,1,0.0094483300,1.2e+09,0.00122097],
["W",1,"R",6,994.260490,0.000990,3,0.0115345000,1.2e+09,-0.00231093]
]

VizieR = pd.DataFrame(data,columns=columns)
VizieR.style

In [5]:
#From MOLAT, Abgrall et al.
#H2 C+  state     dcanjp94
#"v_upper", "J_upper", "v_lower", "J_lower", "A (s-1)", "TR.E.(cm-1)"

columns = ["v_upper", "J_upper", "v_lower", "J_lower", "A", "E"]
data = [
[1,  1,  0,  0, 0.1577E+09, 101457.97],
[1,  2,  0,  1, 0.1360E+09, 101456.88],
[1,  3,  0,  2, 0.1283E+09, 101395.25],
[1,  4,  0,  3, 0.1365E+09, 101271.58],
[1,  5,  0,  4, 0.4860E+07, 101125.54],
[1,  6,  0,  5, 0.6175E+08, 100870.70],
[1,  7,  0,  6, 0.7505E+08, 100577.88]
]
Abgrall = pd.DataFrame(data,columns=columns)
Abgrall.style

## New evaluation of $f$-values follows

In [6]:
def oscillatorStrengthOnDataframe(x):
    J_lower = x['J_lower']
    A = x['A']
    E = x['E']
    return oscillatorStrength(branch="R", J_lower = J_lower, A_coeff = A, E_in_icm = E)


f_coeff = Abgrall.apply( oscillatorStrengthOnDataframe, axis=1 )

Abgrall2 = Abgrall.assign(f = f_coeff.values)
Abgrall2 = Abgrall2.assign(f_VizieR = VizieR['f'].values)
Abgrall2.style

In [7]:
def g2_over_g1(x):
    J_lower = x['J_lower']
    J_upper = x['J_upper']
    
    return ( 2.0*J_upper + 1 )/( 2.0*J_lower + 1 )

g2_over_g1 = Abgrall2.apply(g2_over_g1, axis=1)
Abgrall2 = Abgrall2.assign(g2_over_g1 = g2_over_g1.values)
#Abgrall2.style

In [8]:
def modifyVizier(x):
    return x['g2_over_g1']*x['f_VizieR']

#Abgrall2['g2_over_g1']
f_VizieR_mod = Abgrall2.apply(modifyVizier, axis=1)
Abgrall2 = Abgrall2.assign(f_VizieR_mod = f_VizieR_mod.values)
#Abgrall2.style

In [9]:
def getRatio(x):
    return x['f_VizieR_mod']/x['f']

#Abgrall2['g2_over_g1']
ratio = Abgrall2.apply(getRatio, axis=1)
Abgrall2 = Abgrall2.assign(f_ratio = ratio.values)
Abgrall2.style

# Remarks

In the previous table, the f column is the result of the new evaluation which is different from f_VizieR. It seems most of the discrepancy could be due to a missing $g_2/g_1$ factor, which is more severe for the low $J$ transitions. (Unfortunate, since these are the more intense lines.)

After accounting for the multiplicity factor in f_VizieR_mod = g2_over_g1 x f_VizieR, a discrepancy in the order of 5% still persists as seen from f_ratio = f_VizieR_mod / f.

The $\Gamma$ coefficients in Vizier are also not the latest values and need to be updated.